In [1]:
pwd

'/Users/amlvt225/Code/GitHub/donkey_golf/jupyter_notebooks'

In [2]:
cd ..

/Users/amlvt225/Code/GitHub/donkey_golf


In [3]:
ls

README.md          jupyter_notebooks/ run.py
donkey_golf/       requirements.txt


In [4]:
import donkey_golf as dg

/Users/amlvt225/Code/VirtualEnvs/venv_dgolf/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [5]:
yay = dg.data_things.PullLeaderboard()

2019-04-21 21:47:30,193 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-04-21 21:47:30,208 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-21 21:47:30,485 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056527


In [6]:
yay.run()

2019-04-21 21:47:31,301 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-21 21:47:31,630 - donkey_golf.data_things.tourney_leaderboard - INFO - Tourney current status: finished


In [7]:
yay.calculate_cut_line()

2019-04-21 21:47:34,791 - donkey_golf.data_things.tourney_leaderboard - INFO - Running calculate cut line
2019-04-21 21:47:34,794 - donkey_golf.data_things.tourney_leaderboard - INFO - Determining the current round
2019-04-21 21:47:34,799 - donkey_golf.data_things.tourney_leaderboard - INFO - Determined the current round to be finished
2019-04-21 21:47:34,799 - donkey_golf.data_things.tourney_leaderboard - INFO - Dont need to worry about the cut line now


In [ ]:
yay.determine_current_round()

In [ ]:
yay.run()

In [ ]:
big = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
big.run()

In [ ]:
big.user_df

In [ ]:
import oyaml
import pandas as pd
import logging
import psycopg2

from sqlalchemy import create_engine

from donkey_golf import config

# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

class BaseClass():

    def __init__(self, **kwargs):
        logger.debug("ABOUT TO SET yaml_sql_dict")
        self.conf = config.DGConfig()
        self.yaml_sql_dict = oyaml.load(open(self.conf.yaml_sql_loc), Loader=oyaml.FullLoader)
        self.tourney_id = self.determine_current_tourney_id()

    def run_sql(self, sql):
        '''
        Takes a SQL and connects to database to execute passed SQL
        '''
        #engine = create_engine(self.conf.db_url)
        with psycopg2.connect(self.conf.db_url) as conn:
        #with engine.connect() as conn:
            df = pd.read_sql(sql, conn)

        conn.close()

        return df

    def load_table_to_db(self, df, tablename):
        '''
        Takes a df and tablename and loads it to our database
        '''
        engine = create_engine(self.conf.db_url)

        with engine.connect() as conn:
            df.to_sql(tablename, conn, if_exists='replace', index=False)

    def determine_current_tourney_id(self):
        '''
        Pull each users aggregate score for the current tourney
        '''
        sql = self.yaml_sql_dict.get('determine_current_tourney_id')
        df = self.run_sql(sql)

        return df['value'][0].strip()


In [ ]:
class LoadWorldRankings(BaseClass):
    def __init__(self):
        BaseClass.__init__(self)

    def scrape_world_rankings_data(self):
        logger.info("Scraping world rankings data")
        rankings = pd.read_html('http://www.owgr.com/ranking?pageNo=1&pageSize=All&country=All', header=0)[0]

        keep_cols_rankings = ['This Week', 'Last week', 'End 2018', 'Name', 'Events Played (Actual)']
        rankings = rankings[keep_cols_rankings]
        
        self.data = rankings.copy()
        
    def clean_data(self):
        logger.info("Cleaning data")
        self.data.rename(columns= {
                                'This Week': 'current_rank',
                                'Last week':'lw_rank',
                                'End 2018':'ly_rank',
                                'Name':'player_raw',
                                'Events Played (Actual)':'events_played'},
                                inplace=True)
        self.fix_player_names()
        self.limit_to_top_players()
    
    def fix_player_names(self):
        logger.info("Fixing some players names that dont align between systems")

        # Some of the players names are messed up
        # read correct mappings from YAML file and apply them
        yaml_dict = oyaml.load(open(self.conf.yaml_world_rankings_loc), Loader=oyaml.FullLoader)
        player_map = yaml_dict.get('world_rankings')
        
        self.data['player_cleaned'] = self.data['player_raw'].map(player_map)
        self.data['player'] = self.data['player_cleaned'].combine_first(self.data['player_raw'])

    def limit_to_top_players(self):
        logger.info("Trimming world rankings to only top 400")
        self.data = self.data.loc[self.data['current_rank'] <= 400]
        
    def run(self):
        self.scrape_world_rankings_data()
        self.clean_data()
        self.load_table_to_db(self.data, 'pre_tourney')

In [ ]:
draft = LoadWorldRankings()

In [ ]:
draft.run()

In [ ]:
df = draft.data

In [ ]:
df.head(2)

In [ ]:
df.shape

In [ ]:
df.loc[df['current_rank'] < 400]

In [ ]:
draft.data

In [ ]:
cmon = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
cmon.pull_tourney_leaderboard()

In [ ]:
cmon.data

In [ ]:
import oyaml
import pandas as pd
import logging
import psycopg2

from sqlalchemy import create_engine

from donkey_golf import config


# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

class BaseClass():

    def __init__(self, **kwargs):
        logger.debug("ABOUT TO SET yaml_sql_dict")
        self.conf = config.DGConfig()
        self.yaml_sql_dict = oyaml.load(open(self.conf.yaml_sql_loc), Loader=oyaml.FullLoader)
        self.tourney_id = self.determine_current_tourney_id()

    def run_sql(self, sql):
        '''
        Takes a SQL and connects to database to execute passed SQL
        '''
        #engine = create_engine(self.conf.db_url)
        with psycopg2.connect(self.conf.db_url) as conn:
        #with engine.connect() as conn:
            df = pd.read_sql(sql, conn)

        conn.close()

        return df

    def load_table_to_db(self, df, tablename):
        '''
        Takes a df and tablename and loads it to our database
        '''
        engine = create_engine(self.conf.db_url)

        with engine.connect() as conn:
            df.to_sql(tablename, conn, if_exists='replace', index=False)

    def determine_current_tourney_id(self):
        '''
        Pull each users aggregate score for the current tourney
        '''
        sql = self.yaml_sql_dict.get('determine_current_tourney_id')
        df = self.run_sql(sql)

        return df['value'][0].strip()

In [ ]:
class GameScoreboard(BaseClass):
    def __init__(self):
        BaseClass.__init__(self)

    def aggregate_user_scores(self):
        '''
        Pull each users aggregate score for the current tourney
        '''
        logger.info("Pulling aggregated team score for scoreboard")
        sql = self.yaml_sql_dict.get('aggregate_team_score')
        df = self.run_sql(sql)

        df.sort_values('donkey_score', inplace=True)

        df['rank'] = df['donkey_score'].rank(ascending=True, method='min').astype(int)

        self.data = df.copy()
        
    def determine_ties_in_scoreboard(self):
        '''
        Takes a scoreboard df and determines rank and handles tiesself.

        It's nice to have T2 on the scoreboard instead of multiple "2"s
        '''
        logger.info("Determining where ties exist in the donkey scoreboard")
        df_ties = self.data.copy()

        # Need to create a better rank that handles ties
        df_ties['new_rank'] = df_ties['rank'].copy()

        df_ties = df_ties.groupby(['rank'])['id'].count().reset_index()
        df_ties.loc[df_ties['id'] > 1, 'new_rank'] = "T"

        df_ties.loc[df_ties['new_rank'].notnull(), 'final_rank'] = 'T' + df_ties['rank'].astype(str)
        df_ties.loc[df_ties['new_rank'].isnull(), 'final_rank'] = df_ties['rank'].astype(str)

        self.df_ties = df_ties[['rank','final_rank']]        
    
    def add_ties_to_data(self):
        logger.info("Adding ties to scoreboard data")
        self.data = self.data.merge(self.df_ties,
                                   how='left',
                                   on='rank')
        
    def run(self):
        self.aggregate_user_scores()
        self.determine_ties_in_scoreboard()
        self.add_ties_to_data()

In [ ]:
gs = GameScoreboard()

In [ ]:
gs.run()

In [ ]:
gs.data

In [ ]:
gs.aggregate_user_scores()

In [ ]:
gs.data

In [ ]:
gs.determine_ties_in_scoreboard()

In [ ]:
gs.add_ties_to_data()

In [ ]:
from donkey_golf import data_utils as du

In [ ]:
du.pull_scoreboard()

In [ ]:
class Leaderboard(StockPull):
    
    def __init__(self):
        StockPull.__init__(self)
        self.determine_current_url()
    
    def determine_current_url(self):
        '''
        Returns the specific URL we want to scrape from
        '''        
        self.url_lb = 'http://www.espn.com/golf/leaderboard/_/tournamentId/{}'.format(self.tourney_id)
    
    def scrape_espn_leaderboard(self, url):
        '''
        Scrape the main leaderboard table from the web
        '''
        data_dict = {}
        # Creating leaderboard & rankings dataframes
        leaderboard = pd.read_html(url,header=3)[0]
        leaderboard.columns = leaderboard.columns.str.lower().str.replace(' ','_')

        leaderboard['tourney_id'] = self.tourney_id
        
        self.data = leaderboard.copy()
    
    def determine_tourney_status(self):
        '''
        Look at the leaderboard data and determine current tourney
        status from that data
        '''
        if 'tee_time' in self.data.columns:
            self.tourney_status = 'pre_tourney'
        elif 'fedex_pts' in self.data.columns or 'earnings' in self.data.columns:
            self.tourney_status = 'finished'
            
        elif 'r4' in self.data.columns or 'tot' in self.data.columns:
            self.tourney_status = 'in_progress'
        else:
            self.tourney_status = 'cant_determine'
            
        logger.info(f"Tourney current status: {self.tourney_status}")
        
    def determine_current_round(self):
        '''
        Figure out which round is in progress. Will be helpful for cut
        calculations among other things
        '''
        logger.info("Determining the current round")
        df = self.data.copy()
        
        df = df.loc[df['missed_cut'] == 0]
        
        current_round = None
        
        if 'fedex_pts' in df.columns:
            current_round = 'finished'
        else:
            rounds = ['r1','r2','r3','r4']
            for round_num in rounds:
                if df.loc[df[round_num] == '--'].shape[0] > 1:
                    current_round = round_num                    
        logger.info(f"Determined the current round to be {current_round}")

        return current_round
        
    def add_donkey_game_score(self):
        '''
        Will add a column called donkey score that should
        convert the to_par string to an integer
        '''
        logger.info("Calculating the donkey_score column")
        self.data.loc[self.data['to_par'] == 'E', 'donkey_score'] = 0
        self.data.loc[self.data['to_par'].str[0:1] == '+', 'donkey_score'] = self.data['to_par'].str[1:]
        self.data.loc[self.data['to_par'].str[0:1] == '-', 'donkey_score'] = self.data['to_par']

        # IF they don't have a score of E or one that starts with a '+' or '-'
        # let's assume they didn't make it to the weekend
        self.add_score_for_cut_players()
    
    def add_score_for_cut_players(self):
        '''
        Determines the score for cut players
        and adds a missed_cut field
        '''
        logger.info("Determining score for cut players and adding missed_cut indicator")
        cut_score = self.data.loc[self.data['donkey_score'].notnull(), 'donkey_score'].astype(int).max()
        cut_score = cut_score + 1

        self.data['missed_cut'] = 0
        self.data.loc[self.data['donkey_score'].isnull(), 'missed_cut'] = 1

        self.data.loc[self.data['donkey_score'].isnull(), 'donkey_score'] = cut_score
        
    def add_player_left_indicator(self):
        '''
        This should add a column 'player_left'
        indicating if a player is still in progress
        '''
        logger.info("Determine if the player is active and if he has been cut or not")
        self.data['player_left'] = 1

        self.data.loc[self.data['missed_cut'] == 1, 'player_left'] = 0

        self.data.loc[(self.data['missed_cut'] == 0) &
              (self.data['r4'] != '--' ),
              'player_left'] = 0

    def add_golfer_team_count(self):
        '''
        This should add a column 'team_count'
        indicating the amount of teams a player is on
        '''
        self.golfer_team_count()
        
        logger.debug("Adding golfer team count")
        self.data = self.data.merge(self.golfer_count_by_team,
            how='left',
            on='player')

        self.data['team_count'] = self.data['team_count'].fillna(0.0).astype(int)

    def golfer_team_count(self):
        '''
        Pull each users aggregate score for the current tourney
        '''
        sql = yaml_sql_dict.get('golfer_team_count')
        df = self.run_sql(sql)

        self.golfer_count_by_team = df
    
    def prepare_active_data(self):
        '''
        If we know the data is from after the tourney has started,
        there will be manipulations we need to do
        '''
        logger.info("Running prepare_in_progress_data")
        self.clean_active_cols()
        self.add_donkey_game_score()
        self.add_player_left_indicator()
        self.add_golfer_team_count()

    def clean_active_cols(self):
        '''
        Clean function for the active tourney data
        '''
        self.data = self.data.drop('unnamed:_1', axis=1)
    
    def prepare_pre_tourney_data(self):
        '''
        This will be used if we need to make any updated to the pre_tourney data
        '''
        pass
    
    def clean_data(self):  
        '''
        Depending on the status of the tourney (and what .data will look like)
        we want to prepare/cleanse the data
        '''
        logger.info("Running clean_data function")
        if self.tourney_status in ['in_progress','finished']:
            self.prepare_active_data()
        elif self.tourney_status == 'pre_tourney':
            self.prepare_pre_tourney_data()
            
    def load_leaderboard_to_db(self):
        '''
        Take self.data and load it into the appropriate table based
        on the tourney_status
        '''
        if self.tourney_status == 'pre_tourney':
            logger.info("Loading data into the pre_tourney table")
            self.load_table_to_db(self.data, 'pre_tourney')
        else:
            logger.info("Loading data into the leaderboard table")            
            self.load_table_to_db(self.data, 'leaderboard')
        
    def run(self):
        # Get the leaderboard data
        self.scrape_espn_leaderboard(self.url_lb)
        
        # Determine the tourney status (set an attribute)
        self.determine_tourney_status()
        
        # Clean data - smart enough to determine pre_tourney or in-progress data
        self.clean_data()
        
        # Don't need this right now
        #self.load_leaderboard_to_db()

In [ ]:
fox = Leaderboard()

In [ ]:
fox.run()

In [ ]:
fox.determine_current_round()

In [ ]:
fox.load_leaderboard_to_db()

In [ ]:
fox.run()

In [ ]:
fox.data

In [ ]:
fox.load_leaderboard_to_db()

In [ ]:
fox.data

In [ ]:
fox.determine_current_round()

In [ ]:
fox.data

In [ ]:
class PullLeaderboard(StockPull):
    
    def __init__(self, **kwargs):
        StockPull.__init__(self)
        self.kwargs = kwargs
        if kwargs.get('user_id'):
            self.user_id = kwargs.get('user_id')
        else:
            self.user_id = None
            
    def pull_tourney_leaderboard(self):
        '''
        Pulls leaderboard data for the tourney
        '''

        sql = yaml_sql_dict.get('pull_tourney_leaderboard')
        self.data = self.run_sql(sql)
        self.determine_tourney_status()
        
        if self.user_id and self.tourney_status == 'in_progress':
            self.pull_users_team()
            
            # If the user_df comes back with records, add on_team field
            if not self.user_df.empty:
                self.data = self.data.merge(user_df[['player','id']],
                    how='left',
                    on=['player'])

                self.data['on_team'] = 'No'
                self.data.loc[self.data['id'].notnull(), 'on_team'] = 'Yes'

                self.data = self.data.drop('id', axis=1)

        self.data['team_count'] = self.data['team_count'].astype(int)
    
    def determine_tourney_status(self):
        '''
        Look at the leaderboard data and determine current tourney
        status from that data
        '''
        if 'tee_time' in self.data.columns:
            self.tourney_status = 'pre_tourney'
        elif 'fedex_pts' in self.data.columns or 'earnings' in self.data.columns:
            self.tourney_status = 'finished'
            
        elif 'r4' in self.data.columns or 'tot' in self.data.columns:
            self.tourney_status = 'in_progress'
        else:
            self.tourney_status = 'cant_determine'
            
        logger.info(f"Tourney current status: {self.tourney_status}")
        
    def pull_users_team(self):
        '''
        Pulls leaderboard data for everyone on a given users team
        '''
        
        if self.tourney_status == 'pre_tourney':
            logger.info("Pulling users pre_tourney team")
            sql = yaml_sql_dict.get('users_team_pre_tourney')
            sql = sql.format(self.user_id, conf.tourney_id)
            self.user_df = run_sql(sql)
        else:
            logger.info("Pulling users active tourney team")            
            sql = yaml_sql_dict.get('users_team')
            # Sub in users ID into the SQL
            sql = sql.format(self.user_id, self.tourney_id)
            self.user_df = self.run_sql(sql)

In [ ]:
yay = PullLeaderboard(user_id=2)

In [ ]:
yay.pull_tourney_leaderboard()

In [ ]:
yay.user_df

In [ ]:
yay.data

In [ ]:
yay.pull_tourney_leaderboard()

In [ ]:
yay.tourney_id

In [ ]:
yay.kwargs

In [ ]:
yay.user_id

In [ ]:
yay.pull_users_team()

In [ ]:
yay.pull_tourney_leaderboard()

In [ ]:
yay.data

In [ ]:
yay.pull_tourney_leaderboard()

In [ ]:
yay.data

In [ ]:
fox.tourney_status in ['in_progress','finished']

In [ ]:
fox.data

In [ ]:
df = fox.data

In [ ]:
df

In [ ]:
fox.add_donkey_game_score()

In [ ]:
fox.data

In [ ]:
df = fox.data

In [ ]:
'earnings' in fox.data.columns

In [ ]:
'fedex_pts' in fox.data.columns or 'earnings' in fox.data.columns

In [ ]:
fox.tourney_id

In [ ]:
fox.url_lb

In [ ]:
fox.scrape_espn_leaderboard(fox.url_lb)

In [ ]:
cd donkey_golf/

In [ ]:
ls

In [ ]:
cd all_things_data/

In [ ]:
import sql_classes